## **Semana 1 do Challange**

**Nessa primeira semana, a companhia Alura Voz disponibilizou os dados dos seus clientes em formato json. Preferi não fazer o download desses dados e trabalhar a versão disponível online [nesse link](https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json).**

As atividades da primeira semana são:

1. Entender quais informações o conjunto de dados possui;
1. Analisar quais os tipos de dados;
1. Verificar quais são as inconsistências nos dados;
1. Corrigir as inconsistências.

Além disso, temos algumas atividades opcionais:
- Traduzir as colunas e dados;
- Criar coluna de contas diárias.

### Importando as bibliotecas

In [30]:
import requests
import json
import pandas as pd

### Importando os dados

#### Lendo o arquivo Json e conhecendo os dados

In [31]:
base_de_dados_ulr = 'https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json'

req = requests.get(base_de_dados_ulr)
response = json.loads(req.text)
response[0] # visualizando a variável response é possível perceber que temos uma lista com dicionários dentro de dicionários

{'customerID': '0002-ORFBO',
 'Churn': 'No',
 'customer': {'gender': 'Female',
  'SeniorCitizen': 0,
  'Partner': 'Yes',
  'Dependents': 'Yes',
  'tenure': 9},
 'phone': {'PhoneService': 'Yes', 'MultipleLines': 'No'},
 'internet': {'InternetService': 'DSL',
  'OnlineSecurity': 'No',
  'OnlineBackup': 'Yes',
  'DeviceProtection': 'No',
  'TechSupport': 'Yes',
  'StreamingTV': 'Yes',
  'StreamingMovies': 'No'},
 'account': {'Contract': 'One year',
  'PaperlessBilling': 'Yes',
  'PaymentMethod': 'Mailed check',
  'Charges': {'Monthly': 65.6, 'Total': '593.3'}}}

#### Criando o DataFrame

In [32]:
df = pd.json_normalize(response)
print(f'O DataFrame tem {df.shape[0]} linhas e {df.shape[1]} colunas')
df.head(2)

O DataFrame tem 7267 linhas e 21 colunas


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4


**Renomeando as colunas**

In [33]:
colunas = []
for key, item in response[0].items():
    if type(item) is dict:
        for key_1, item_1 in item.items():
            if type(item_1) is dict:
                for key_2, item_2 in item_1.items():
                    colunas.append(key_2)
            else:
                colunas.append(key_1)
    else:
        colunas.append(key)

df.columns = colunas
df.rename(columns = {'Monthly' : 'Charges_Monthly', 'Total' : 'Charges_Total'}, inplace = True)

df.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges_Monthly',
       'Charges_Total'],
      dtype='object')

### Analisando os tipos de dados

Informações das variáveis

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


O dataset contém 20 variáveis. Sendo a maioria delas do tipo object, e a coluna ternure e coluna Charges_Monthly do tipo float64. Porém, a coluna Charges_Total deveria ser reconhecida como do tipo float64, esta correção é feita na próxima seção.

### Verificando Inconsistências nos dados

In [35]:
for i in list(df.columns):
    print(f'{i}{df[i].unique()}')

customerID['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
Churn['No' 'Yes' '']
gender['Female' 'Male']
SeniorCitizen[0 1]
Partner['Yes' 'No']
Dependents['Yes' 'No']
tenure[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
PhoneService['Yes' 'No']
MultipleLines['No' 'Yes' 'No phone service']
InternetService['DSL' 'Fiber optic' 'No']
OnlineSecurity['No' 'Yes' 'No internet service']
OnlineBackup['Yes' 'No' 'No internet service']
DeviceProtection['No' 'Yes' 'No internet service']
TechSupport['Yes' 'No' 'No internet service']
StreamingTV['Yes' 'No' 'No internet service']
StreamingMovies['No' 'Yes' 'No internet service']
Contract['One year' 'Month-to-month' 'Two year']
PaperlessBilling['Yes' 'No']
PaymentMethod['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer

Verificando se há ID´s repetidos

In [36]:
if df.customerID.nunique() == df.shape[0]:
    print('Não existem IDs duplicados')

Não existem IDs duplicados


Na coluna Churn existe incosistência, pois algumas células estão vazias.

In [37]:
df.Churn.unique()

array(['No', 'Yes', ''], dtype=object)

Verificando se há inconsistência quanto as colunas referentes a serviço telefônico:
PhoneService,         MultipleLines. Neste caso uma inconsistência seria se um cliente que não têm registro telefônico não estivesse classificdao como "No phone service"

In [38]:
quantidade_clientes_sem_linha_telefonica = len(df.loc[df.PhoneService == "No"])
quantidade_clientes_sem_mutiplas_linhas = len(df.loc[df.MultipleLines == 'No phone service'])

if quantidade_clientes_sem_linha_telefonica == quantidade_clientes_sem_mutiplas_linhas:
    print('Não existe inconsistência aparente')
else:
    print('Existe inconsistência')

Não existe inconsistência aparente


Da mesma forma será verificada a possível inconsistência relacionada ao serviço de internet. Clientes sem serviço de internet não podem ter os serviços relacionados: OnlineSecurity, OnlineBackup, DeviceProtection, TechSuport, StreamingTV, StreamingMovies.

In [39]:
quantidade_clientes_sem_internet = len(df.loc[df.InternetService == 'No'])

serviço_de_internet = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for i in serviço_de_internet:
    if len(df[df[i] =='No internet service']) == quantidade_clientes_sem_internet:
        print(f'{i} não há inconsistência')
    else:
        print(f'{i} há inconsistência')


OnlineSecurity não há inconsistência
OnlineBackup não há inconsistência
DeviceProtection não há inconsistência
TechSupport não há inconsistência
StreamingTV não há inconsistência
StreamingMovies não há inconsistência


Não foram encontradas incosistências

A colunas Charges_Total está como object e não como Float

In [40]:
df.Charges_Total.unique()

array(['593.3', '542.4', '280.85', ..., '742.9', '4627.65', '3707.6'],
      dtype=object)

Outra incosistência possível seria valores negativos em algumas das variáveis numéricas. A seguir é verificado isto.

In [41]:
df.describe()

,SeniorCitizen,tenure,Charges_Monthly
count,7267.000000,7267.000000,7267.000000
mean,0.162653,32.346498,64.720098
std,0.369074,24.571773,30.129572
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.425000
50%,0.000000,29.000000,70.300000
75%,0.000000,55.000000,89.875000
max,1.000000,72.000000,118.750000


Em todas as variáveis numéricas o valor mínimo é zero portanto não há a inconsistência de números negativos. Porém, ainda será necessario ajustar a coluna "Charges_Total" que veio como string, Após a transformação dos dados de string para float será verificado novamente se há valores estranhos na coluna.

### Corrigindo Incosistências

Removendo os valores nulos da coluna Churn.

In [42]:
df.Churn.value_counts()

No     5174
Yes    1869
        224
Name: Churn, dtype: int64

In [43]:
df = df[df.Churn != '']

In [44]:
df.Churn.unique()

array(['No', 'Yes'], dtype=object)

Transformando a colunas Charges_Total em float64

In [45]:
df['Charges_Total'] = df['Charges_Total'].apply(pd.to_numeric, errors='coerce')
df.Charges_Total.dtypes

dtype('float64')

Verificando quantidade de registros nulos

In [46]:
df.Charges_Total.isna().sum()

11

Olhando os registro nulos

In [47]:
df[df.Charges_Total.isna()]

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN


Todos os registros com Charges_total nulo, tem tenure = 0. Sendo assim, os clientes estão em seu primeiro mês de contrato e não gastaram nada ainda, logo o Charges_total deve ser 0

Transformando os valores nulos em 0

In [48]:
df.Charges_Total.fillna(0, inplace = True)
print(f'Há {df.Charges_Total.isna().sum()} valores nulos')

Há 0 valores nulos


### Traduzindo os dados

#### Traduzindos os dados das colunas

Tradução coluna SeniorCitizen para 'Yes' e "No'

In [49]:
dict_yes_no = {1 : 'Yes' , 0 : 'No'}

df['SeniorCitizen'].replace(dict_yes_no, inplace=True)

Tradução colunas 'yes', 'no', 'No phone service', 'No internet service' para variáveis categoricas binárias.

In [50]:
dict_yes_no = {'Yes' : 'Sim' , 'No' : 'Nao' , 'No phone service' : 'Sem servico de telefone' , 'No internet service' : 'Sem servico de internet' }

colunas_yes_no = ['Churn', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                     'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                     'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'SeniorCitizen']

for coluna in colunas_yes_no:
    df[coluna].replace(dict_yes_no, inplace=True)

Tradução gêneros 'female' e 'male para variáveis categoricas binárias. Sendo 1 Maculino e 0 Feminino

In [51]:
dict_gender = {'Male': 'Homem' , 'Female': 'Mulher'}

df.gender.replace(dict_gender, inplace=True)
df.gender.unique()

array(['Mulher', 'Homem'], dtype=object)

Tradução contratos

In [52]:
contratos = {'Month-to-month' : 'Mensal', 'Two year': 'Bienal', 'One year': 'Anual'}
df.Contract.replace(contratos, inplace=True)
df.Contract.unique()

array(['Anual', 'Mensal', 'Bienal'], dtype=object)

Tradução dos métodos de pagamento.

In [53]:
metodo_pagamento = {'Electronic check': 'Cheque eletrônico', 'Mailed check':'Cheque',
                    'Bank transfer (automatic)': 'Transferência bancária automático',
                    'Credit card (automatic)': 'Cartão de crédito automático'}

df.PaymentMethod.replace(metodo_pagamento, inplace=True)
df.PaymentMethod.unique()

array(['Cheque', 'Cheque eletrônico', 'Cartão de crédito automático',
       'Transferência bancária automático'], dtype=object)

Conferindo as mudanças

In [54]:
for i in list(df.columns):
    print(f'{i}{df[i].unique()}')

customerID['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
Churn['Nao' 'Sim']
gender['Mulher' 'Homem']
SeniorCitizen['Nao' 'Sim']
Partner['Sim' 'Nao']
Dependents['Sim' 'Nao']
tenure[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]
PhoneService['Sim' 'Nao']
MultipleLines['Nao' 'Sim' 'Sem servico de telefone']
InternetService['DSL' 'Fiber optic' 'Nao']
OnlineSecurity['Nao' 'Sim' 'Sem servico de internet']
OnlineBackup['Sim' 'Nao' 'Sem servico de internet']
DeviceProtection['Nao' 'Sim' 'Sem servico de internet']
TechSupport['Sim' 'Nao' 'Sem servico de internet']
StreamingTV['Sim' 'Nao' 'Sem servico de internet']
StreamingMovies['Nao' 'Sim' 'Sem servico de internet']
Contract['Anual' 'Mensal' 'Bienal']
PaperlessBilling['Sim' 'Nao']
PaymentMethod['Cheque' 'Cheque eletrônico' 'Cartão d

#### Traduzindo o nome das colunas

In [55]:
dict_colunas_traduzido = {
'customerID':'ID_cliente', 
'Churn' : 'Evasao',
'gender' :'Genero', 
'SeniorCitizen' : 'Cidadao_Senior', 
'Partner' : 'Parceiro',
'Dependents' : 'Dependentes', 
'tenure' : 'Tempo_de_contrato', 
'PhoneService' : 'Servico_de_telefone',
'MultipleLines' : 'Linhas_multiplas',
'InternetService': 'Servico_de_internet', 
'OnlineSecurity' :'Segurnaca_online', 
'OnlineBackup' : 'Backup_online', 
'DeviceProtection' : 'Protecao_dipositivo',
'TechSupport' : 'Suporte_tecnico',
'StreamingTV' : 'Streaming_TV',
'StreamingMovies' : 'Streaming_filmes',
'Contract' : 'Tipo_de_contrato',
'PaperlessBilling' : 'Fatura_online' ,
'PaymentMethod' : 'Metodo_de_pagamento',
'Charges_Monthly' : 'Total_servico_mes',
'Charges_Total' : 'Total_gasto'
}

df.rename(columns = dict_colunas_traduzido, inplace = True)
df.columns

Index(['ID_cliente', 'Evasao', 'Genero', 'Cidadao_Senior', 'Parceiro',
       'Dependentes', 'Tempo_de_contrato', 'Servico_de_telefone',
       'Linhas_multiplas', 'Servico_de_internet', 'Segurnaca_online',
       'Backup_online', 'Protecao_dipositivo', 'Suporte_tecnico',
       'Streaming_TV', 'Streaming_filmes', 'Tipo_de_contrato', 'Fatura_online',
       'Metodo_de_pagamento', 'Total_servico_mes', 'Total_gasto'],
      dtype='object')

### Criando a coluna de Total_servico_dia

In [56]:
df.insert(19, 'Total_servico_dia', round(df.Total_servico_mes / 30, 2), True)

In [57]:
df.head(2)

,ID_cliente,Evasao,Genero,Cidadao_Senior,Parceiro,Dependentes,Tempo_de_contrato,Servico_de_telefone,Linhas_multiplas,Servico_de_internet,...,Protecao_dipositivo,Suporte_tecnico,Streaming_TV,Streaming_filmes,Tipo_de_contrato,Fatura_online,Metodo_de_pagamento,Total_servico_dia,Total_servico_mes,Total_gasto
0,0002-ORFBO,Nao,Mulher,Nao,Sim,Sim,9,Sim,Nao,DSL,...,Nao,Sim,Sim,Nao,Anual,Sim,Cheque,2.19,65.6,593.3
1,0003-MKNFE,Nao,Homem,Nao,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Sim,Mensal,Nao,Cheque,2.00,59.9,542.4


### Exportando o arquivo em CSV

In [58]:
df.to_csv('D:\OneDrive\Pessoal\Material de Estudo\ALURA\Desafios\Alura_Challenge_Data_Science\Semana 01\dados limpos.csv', index=False)